# Notebook 8: encoding of variables

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd

import data_processing_helper as dp
import practical_functions as pf

import warnings
warnings.filterwarnings("ignore")

/home/lavinia/.local/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
df_data = pd.read_csv("../Data/merged_30_04.csv")
df_extended = pd.read_csv("../Data/merged_5_05_extended.csv")
df_data.drop(columns = "id", inplace = True)

df_extended## Treating the extended file 

Steps: 
- bound the variable to end with its corresponding date


In [4]:
def bound_dates(df_extended, list_dates):
    df_concat = []
    df_wind = []
    df_rad = []
    df_rad_clear = []
    df_DHW = []
    df_SST = []
    end_col_wind = df_extended.columns.get_loc([col for col in df_extended.columns if "wind_" in col][-1]) + 1
    end_col_rad = df_extended.columns.get_loc([col for col in df_extended.columns if "rad_solar_2018-12" in col][0]) +1
    end_col_clear_rad = df_extended.columns.get_loc([col for col in df_extended.columns if "clear_rad_solar_" in col][-1]) +1
    end_col_DHW = df_extended.columns.get_loc([col for col in df_extended.columns if "DHW_" in col][-1])+ 1
    end_col_SST = df_extended.columns.get_loc([col for col in df_extended.columns if "SST_" in col][-1]) + 1
    for date in list_dates:
        df_date = df_extended[df_extended["surveydate"] == date]
        lend_date = [col for col in df_extended.columns if date in col]
        
        st_wind = df_extended.columns.get_loc(lend_date[0]) + 1
        st_rad = df_extended.columns.get_loc(lend_date[1]) + 1
        st_clear_rad = df_extended.columns.get_loc(lend_date[2])+1 
        st_DHW = df_extended.columns.get_loc(lend_date[3]) + 1
        st_SST = df_extended.columns.get_loc(lend_date[4]) +1
        
    
        
        df = df_date.iloc[:, np.r_[:st_wind, end_col_wind:st_rad, end_col_rad:st_clear_rad, end_col_clear_rad:st_DHW, end_col_DHW:st_SST]]
        df_wind.append(df_date.iloc[:, np.r_[:st_wind]])
        df_rad.append(df_date.iloc[:, np.r_[end_col_wind:st_rad]])
        df_rad_clear.append(df_date.iloc[:, np.r_[end_col_rad:st_clear_rad]])
        df_DHW.append(df_date.iloc[:, np.r_[end_col_clear_rad:st_DHW]])
        df_SST.append(df_date.iloc[:, np.r_[end_col_DHW:st_SST]])
        
        df_concat.append(df)
        
    df = pd.concat(df_wind).join(pd.concat(df_rad)).join(pd.concat(df_rad_clear)).join(pd.concat(df_DHW)).join(pd.concat(df_SST))
        
    return df
        

        
    

In [5]:
list_dates = df_extended["surveydate"].unique()

In [6]:
# In this dataframe, the end dates for variables is in the start of the survey date (we don't know the future )
df_date_match= bound_dates(df_extended, list_dates)

In [7]:
df_date_match.to_csv("../Data/df_extended_correct_end_dates_17_05.csv", index = False)

## Note
In this dataframe the month in which the survey was taken is taken into account

In [8]:

df_extended = df_date_match

In [9]:
df_extended.columns[10:40]

Index(['geometry', 'wind_speed', 'Chlor', 'solar_rad', 'solar_rad_clear_sky',
       'elevation', 'id', 'DHW', 'SST', 'par', 'closest_port', 'year',
       'distance_urban', 'dist_marine_protection', 'Population',
       'Population_growth', 'agr_type', 'Broad_type', 'dist_agr_AUS',
       'mine_area_per_call', 'distance_to_mine_area', 'mine_area_per_cell',
       'nigh_avg_2012', 'nigh_avg_2013', 'nigh_avg_2014', 'nigh_avg_2015',
       'nigh_avg_2016', 'nigh_avg_2017', 'nigh_avg_2018', 'fish_hours_2012'],
      dtype='object')

In [10]:
[col for col in df_extended.columns if "nigh" in col]

['nigh_avg_2012',
 'nigh_avg_2013',
 'nigh_avg_2014',
 'nigh_avg_2015',
 'nigh_avg_2016',
 'nigh_avg_2017',
 'nigh_avg_2018']

# Make meaningful variables out of the data

#### Working with wind data
- Take mean/min/max/range two months before, 6 months before, 12 months before, 10 years, 20 years
--> Not very valuable I think due to the resolution :(

Improvement is to use hourly around the time of survey
https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-land?tab=overview

In [11]:
## Here we inclue the month of the survey
# By setting id_start you can give a range to which years you want 

def process_meaningful_data(df, name_var, date):
    
    id_end = df.columns.get_loc([col for col in df.columns if format(f"{name_var}{date}") in col][0])
    
    id_start = df.columns.get_loc([col for col in df.columns if format(f"{name_var}") in col][0])
    
    
    df_meaning = []
    
    """
    month_before = 2
    
    if (format(f"{name_var}") in df_extended.columns[id_end-month_before]):
        df_meaning.append(process_all(df, name_var, month_before, id_end))
    month_before = 6
    if (format(f"{name_var}") in df_extended.columns[id_end-month_before]):
        df_meaning.append(process_all(df, name_var, month_before, id_end))
        """
    # 1 year before (included)
    month_before = 12
    if (format(f"{name_var}") in df_extended.columns[id_end-month_before]):
        df_meaning.append(process_all(df, name_var, month_before, id_end))
        
    # 10 years before (included)
    month_before = 12*10
    if (format(f"{name_var}") in df_extended.columns[id_end-month_before]):
        df_meaning.append(process_all(df, name_var, month_before, id_end))

    # 20 years before (included)
    month_before = 12*20
    
    if (format(f"{name_var}") in df_extended.columns[id_end-month_before]):
        df_meaning.append(process_all(df, name_var, month_before, id_end))
    
    # 2 years before but excluding survey year (year_bef)
    month_before = 12*2
    year_bef = 12
    if (format(f"{name_var}") in df_extended.columns[id_end - year_bef -month_before]):
        
        df_meaning.append(process_all(df, name_var, month_before, id_end-year_bef, exclude = True))
    
    
    # all years before but excluding survey year
    if (format(f"{name_var}") in df_extended.columns[id_end - year_bef]):
        
        df_meaning.append(process_all(df, name_var, month_before, id_end-year_bef, id_start))
    
    
    if (not df_meaning):
        # empty concatenation
        return 
    
    return pd.concat(df_meaning, axis = 1)

    

def process_all(df, name_var, month_before, id_end, id_start = False, exclude = False):
    
    if(exclude != False):
        df[format(f"{name_var}2year_exclude_avg")] = monthly_average(df, month_before, id_end, id_start)
        df[format(f"{name_var}2year_exclude_min")] = min_monthly(df, month_before, id_end, id_start)
        df[format(f"{name_var}2year_exclude_max")] = max_monthly(df, month_before, id_end, id_start)
        df[format(f"{name_var}2year_exclude_range")] = range_monthly(df, month_before, id_end, id_start) 
        merge_ = pd.concat([df[format(f"{name_var}2year_exclude_avg")],df[format(f"{name_var}2year_exclude_min")], df[format(f"{name_var}2year_exclude_max")], df[format(f"{name_var}2year_exclude_range")]], axis = 1)
        return merge_
        
    
    elif (id_start != False):
        
        df[format(f"{name_var}all_year_avg")] = monthly_average(df, month_before, id_end, id_start)
        df[format(f"{name_var}all_year_min")] = min_monthly(df, month_before, id_end, id_start)
        df[format(f"{name_var}all_year_max")] = max_monthly(df, month_before, id_end, id_start)
        df[format(f"{name_var}all_year_range")] = range_monthly(df, month_before, id_end, id_start) 
        merge_ = pd.concat([df[format(f"{name_var}all_year_avg")],df[format(f"{name_var}all_year_min")], df[format(f"{name_var}all_year_max")], df[format(f"{name_var}all_year_range")]], axis = 1)
        return merge_
            
    elif (id_start == False):
        
        df[format(f"{name_var}{month_before}_monthly_avg")] = monthly_average(df, month_before, id_end)
        df[format(f"{name_var}{month_before}_monthly_min")] = min_monthly(df, month_before, id_end)
        df[format(f"{name_var}{month_before}_monthly_max")] = max_monthly(df, month_before, id_end)
        df[format(f"{name_var}{month_before}_monthly_range")] = range_monthly(df, month_before, id_end) 
        merge_ = pd.concat([df[format(f"{name_var}{month_before}_monthly_avg")],df[format(f"{name_var}{month_before}_monthly_min")], df[format(f"{name_var}{month_before}_monthly_max")], df[format(f"{name_var}{month_before}_monthly_range")]], axis = 1)
        
        return merge_
    
    
def monthly_average(df, number_of_months, id_end, id_start = False):
    if (id_start != False):
        return df[df.columns[id_start:id_end+1]].mean(axis = 1)
    
    # does include the month of interest
    return df[df.columns[id_end-number_of_months+1:id_end+1]].mean(axis = 1)
def min_monthly(df, number_of_months, id_end, id_start = False):
    if (id_start != False):
        return df[df.columns[id_start:id_end+1]].mean(axis = 1)
    return df[df.columns[id_end-number_of_months+1:id_end+1]].min(axis = 1)
def max_monthly(df, number_of_months, id_end, id_start = False):
    if (id_start != False):
        return df[df.columns[id_start:id_end+1]].mean(axis = 1)
    return df[df.columns[id_end-number_of_months+1:id_end+1]].max(axis = 1)
def range_monthly(df, number_of_months, id_end, id_start = False):
    if (id_start != False):
        return df[df.columns[id_start:id_end+1]].mean(axis = 1)
    min_ = min_monthly(df, number_of_months, id_end)
    max_ = max_monthly(df, number_of_months, id_end)
    return max_ - min_

def combine_processing_dates(df, name_var, list_dates):
    
    list_df = []
    
    for date in list_dates:
        df_date = df[df["surveydate"] == date]
       
        list_df.append(process_meaningful_data(df_date, name_var, date))
        
    
    
    return pd.concat(list_df)
        
        
def process_all_variables(df, list_dates, list_var):        
    
    list_all_df = []
    
    for name_var in list_var:
        list_all_df.append(combine_processing_dates(df, name_var, list_dates))
        
        
    
    return pd.concat(list_all_df, axis = 1)
    
    
    

In [12]:
names_variables = ["wind_", "solar_", "clear_rad_solar_", "SST_", "DHW_"]
# names_variables = ["wind_"]


In [13]:
col_drop = ['lat', 'lon', 'transectid',  'pr_hard_coral', 'pr_algae',
       'pr_soft_coral', 'pr_oth_invert', 'pr_other', 'country', 'geometry',
       'wind_speed', 'Chlor', 'solar_rad', 'solar_rad_clear_sky', 'elevation',
       'id', 'DHW', 'SST', 'par', 'closest_port', 'year', 'distance_urban',
       'dist_marine_protection', 'Population', 'Population_growth', 'agr_type',
       'Broad_type', 'dist_agr_AUS', 'mine_area_per_call',
       'distance_to_mine_area', 'mine_area_per_cell', 'nigh_avg_2012',
       'nigh_avg_2013', 'nigh_avg_2014', 'nigh_avg_2015', 'nigh_avg_2016',
       'nigh_avg_2017', 'nigh_avg_2018', 'fish_hours_2012', 'fish_hours_2014',
       'fish_hours_2017', 'fish_hours_2016', 'distance_agr_land']

In [14]:
df_combine = df_extended.drop(columns = col_drop)

In [15]:

Combine_useful_variables = process_all_variables(df_combine, list_dates, names_variables)

In [16]:
# starts with wind, ends with SST 
start_remove = df_extended.columns.get_loc([col for col in df_extended.columns if "wind_" in col][1])
end_remove = df_extended.columns.get_loc([col for col in df_extended.columns if "SST_" in col][-1]) + 1

In [17]:
df_extended.drop(columns = df_extended.columns[start_remove:end_remove],inplace = True)

In [18]:
df_extended = df_extended.join(Combine_useful_variables)

In [20]:
df_extended

,lat,lon,transectid,surveydate,pr_hard_coral,pr_algae,pr_soft_coral,pr_oth_invert,pr_other,country,...,DHW_240_monthly_max,DHW_240_monthly_range,DHW_2year_exclude_avg,DHW_2year_exclude_min,DHW_2year_exclude_max,DHW_2year_exclude_range,DHW_all_year_avg,DHW_all_year_min,DHW_all_year_max,DHW_all_year_range
0,-18.833291,147.651632,11002,2012-10,0.1586,0.6852,0.0978,0.0216,0.0366,AUS,...,2.772857,2.772857,0.317211,0.0,1.61,1.61,0.217276,0.217276,0.217276,0.217276
1,-18.806353,147.669078,11001,2012-10,0.1141,0.6164,0.0497,0.0024,0.2173,AUS,...,2.772857,2.772857,0.317211,0.0,1.61,1.61,0.217276,0.217276,0.217276,0.217276
2,-18.665839,147.722256,11004,2012-10,0.0331,0.8568,0.0230,0.0068,0.0805,AUS,...,2.772857,2.772857,0.317211,0.0,1.61,1.61,0.217276,0.217276,0.217276,0.217276
3,-18.656207,147.724072,11003,2012-10,0.0485,0.9070,0.0225,0.0098,0.0121,AUS,...,8.600323,8.600323,0.660255,0.0,2.98,2.98,0.407872,0.407872,0.407872,0.407872
4,-18.590921,147.565032,11005,2012-10,0.4016,0.4756,0.0185,0.0046,0.0998,AUS,...,8.600323,8.600323,0.660255,0.0,2.98,2.98,0.407872,0.407872,0.407872,0.407872
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
411,-8.221244,125.530967,30018,2014-07,0.1706,0.6726,0.0805,0.0244,0.0520,TLS,...,1.300000,1.300000,0.222979,0.0,1.30,1.30,0.109572,0.109572,0.109572,0.109572
412,-8.211822,125.622274,30014,2014-07,0.1706,0.5924,0.1170,0.0689,0.0510,TLS,...,1.300000,1.300000,0.222979,0.0,1.30,1.30,0.109572,0.109572,0.109572,0.109572
413,-8.183781,125.638740,30013,2014-07,0.1273,0.7098,0.0633,0.0483,0.0513,TLS,...,1.300000,1.300000,0.222979,0.0,1.30,1.30,0.109572,0.109572,0.109572,0.109572
414,-8.538504,125.606825,30026,2014-08,0.1017,0.5661,0.2066,0.0140,0.1117,TLS,...,1.300000,1.300000,0.222979,0.0,1.30,1.30,0.109251,0.109251,0.109251,0.109251


In [21]:
df_extended.to_csv("../Data/df_all_17_05.csv", index = False)

In [22]:
df_extended = pd.read_csv("../Data/df_all_17_05.csv")
dp.geo_loads(df_extended)
df_extended = pf.make_geo_frame(df_extended)
